In [13]:
#Preamble
import ipywidgets as widgets
import io
import os
import pandas as pd
import requests
from matplotlib.pyplot import *
import rpy2
import rpy2.robjects as robjects

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)




# **Sample Analysis**
### Introduction
Add some instructions here

### **Step #0:** Lets get some census data to compare to

Select the CSV with the census race data for your town, and press "Load"

In [15]:
robjects.r('''
        # create a function `fetchCensusRaceData`
        fetchCensusRaceData <- function(state,county) {
            c(state,county)
        }
        ''')
r_FetchCensusRaceData = robjects.r['fetchCensusRaceData']

In [ ]:
%matplotlib inline


selectCensusDataMethod = widgets.RadioButtons(
    options=['I have a CSV', 'Fetch census data for me'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Do you already have demographic data?',
)

def load_data0():
    global townRace
    townRaceRaw = pd.read_csv(os.path.join("data/",fileSelect0.value),index_col=[0])
    townRaceRaw.columns=['Population']
    townRaceRaw['Population']=townRaceRaw['Population'].apply(lambda s: s.replace(',',''))
    townRaceRaw['Population']=townRaceRaw['Population'].astype('int')
    totalTownPopulation = townRaceRaw.filter(like='Total',axis='index').iloc[0,0]
    townRace = pd.DataFrame([["White",townRaceRaw.filter(like='White',axis='index')['Population'].sum()],
                          ["Black or African American", townRaceRaw.filter(like='Black',axis='index')['Population'].sum()],
                          ["American Indian or Alaskan Native", townRaceRaw.filter(like='American Indian',axis='index')['Population'].sum()],
                          ["Asian", townRaceRaw.filter(like='Asian',axis='index')['Population'].sum()],
                          ["Native Hawaiian or Other Pacific Islander",townRaceRaw.filter(like='Native Hawaiian',axis='index')['Population'].sum()],
                          ["Some Other Race", townRaceRaw.filter(like='Other',axis='index')['Population'].sum()]])
    townRace.columns=['Race','Population']
    townRace['Percentage']=townRace['Population']/totalTownPopulation
    
    with outputWidget0:
        display("Successfully loaded the datafile " + fileSelect0.value )
        display("The total population of your town is " + str(totalTownPopulation))
        display(townRace)
        townRace.plot.bar(x='Race',y='Percentage')
        show()

outputWidget0 = widgets.Output(layout={'border': '1px solid black'})
fileSelect0 = widgets.Dropdown(
    options= os.listdir('data/'),
    disabled=False
)
loadButton0 = widgets.Button(description='Load data file')
loadButton0.on_click(lambda b: load_data0())

def selection(change):
    outputWidget0.clear_output()
    with outputWidget0:
        display("You have chosen "+selectCensusDataMethod.value)
        if selectCensusDataMethod.value == 'I have a CSV':
            display(fileSelect0)
            display(loadButton0)
        if selectCensusDataMethod.value == 'Fetch census data for me':
            display(r_FetchCensusRaceData("AL","Mobile"))
            
selectCensusDataMethod.observe(selection, names='value')
#Make sure our event handler runs the first time 
selection(0)

display(selectCensusDataMethod)
display(outputWidget0)

### **Step #1:** Load the police data

Select your police datafile from the dropdown, and then press load.
You may need to upload your datafile to the "data" directory on the left.

In [4]:
outputWidget = widgets.Output(layout={'border': '1px solid black'})

fileSelect = widgets.Dropdown(
    options= os.listdir('data/'),
    disabled=False
)
display(fileSelect)
loadButton = widgets.Button(
    description='Load data file',
)

outputWidget3 = widgets.Output(layout={'border': '1px solid black'})
raceDropDown=widgets.Dropdown()
def load_data():
    global arrests
    arrests = pd.read_csv(os.path.join("data/",fileSelect.value),index_col=[0])
    global raceDropDown
    raceDropDown.options=arrests.columns
    with outputWidget3:
        display(raceDropDown)
    
    with outputWidget:
        display("Successfully loaded the datafile " + fileSelect.value )

loadButton.on_click(lambda b: load_data())
display(loadButton,outputWidget)

Dropdown(options=('Durham_arrests_charges.csv', 'Durham_census_race_table.csv'), value='Durham_arrests_charges…

Button(description='Load data file', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

<IPython.core.display.Javascript object>

Let's take a quick look at the data.  Select how many rows you would like to preview.

In [5]:
outputWidget2 = widgets.Output(layout={'border': '1px solid black'})
# Take a first look:
pd.set_option("display.max_rows", None)
numRows = widgets.Dropdown(
    options=['5', '10', '15', '20'],
    value='5',
    description='Number:',
    disabled=False,
)
display(numRows)
def preview_data(change):
    outputWidget2.clear_output()
    with outputWidget2:
        display(arrests.sample(int(numRows.value)))
numRows.observe(preview_data, names='value')
display(outputWidget2)

Dropdown(description='Number:', options=('5', '10', '15', '20'), value='5')

Button(description='Preview', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

### **Step #2:** Which type of data do we have

In [6]:
#outputWidget3 = widgets.Output(layout={'border': '1px solid black'})
#layout = widgets.Layout(width='auto', height='40px') #set width and height
#columnsButton = widgets.Button(
    #description='Click here to continue',
    #layout=layout,
#)
#def displayColumns():
    #with outputWidget3:
        ##display(arrests.columns)
        #global raceDropDown
        #raceDropDown = widgets.Dropdown(
            #options=arrests.columns,
            #disabled=False,
        #)
        #display(raceDropDown)
#columnsButton.on_click(lambda b: displayColumns())
#



Button(description='Display column names', layout=Layout(height='40px', width='auto'), style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Which of these columns represents a demographic category of interest, like race?

In [ ]:
%matplotlib inline
#display(columnsButton,outputWidget3)
display(outputWidget3)

def raceDropdownChange(change):
    with outputWidget3:
        arrests[raceDropDown.value].value_counts(normalize=True).plot.bar()
        show()
raceDropDown.observe(raceDropdownChange,names='value')

In [ ]:
outputWidget8 = widgets.Output(layout={'border': '1px solid black'})
uploader = widgets.FileUpload()

def on_upload_change(change):
    global test
    test=pd.read_csv(io.BytesIO(uploader.value[-1].content))
    with outputWidget8:
        display("File uploaded")
        display("Loading file"+uploader.value[-1].name)
        display(test.sample(5))

uploader.observe(on_upload_change, names='value')

display(uploader, outputWidget8)

